# Optimizing Online 5g Machine-Learning with Nsight Compute

## 02 Running the APSM code

We start by building the APSM code used in this tutorial. While the code is compiling, we can learn what it is about.

The example is cmake-based. Run the below commands to build the application. While the code is building, let's look at a short introduction about APSM. For the full details, watch the [NOMA on the GPU in Real Time](https://gtc21.event.nvidia.com/media/NOMA%20on%20the%20GPU%20in%20Real%20Time%20%5BS31435%5D/1_5c8r121y) GTC talk.

In [ ]:
!mkdir /dli/task/ncu/apsm/cpp/build
%cd /dli/task/ncu/apsm/cpp/build
!cmake ..
!make clean && make -j

`APSM (Adaptive Projected Subgradient Method)` is a wireless communication code. In the context of this lab, a communication system is responsible for the conversion between bits and symbols and analog signals, amplified and wirelessly transmitted over antennas. The receiving end has to do the inverse process: receive the signals, process them, `detect the symbols` and map them back to the bits. Modeling the wireless channel is hard because it has losses, noise, interference and is highly dynamic.

<img src="images/apsm_01.png">

Let’s now have a closer look at the modulation and demodulation stages with a brief example. Here we have a Quad Phase Shift Key (QPSK) modulation scheme. We encode two bits into each symbol, so we get four possible symbols (from the combinations 00, 01, 10, 11). We show the symbols arranged in a constellation diagram, which is just an easy and useful way to display them.

<img src="images/apsm_02.png" width="700">

On the left, we have the transmitted symbols for each of the two users. On the right, we can see what we receive after going over a channel. We can see how the received samples form a pattern similar to the transmitted pattern, but smaller, rotated and ”fuzzier”, as each point is somewhat off-center. As we send more and more symbols (we can see 16 rough locations), they create a cloud around some center.

We want to be able to map (assign) each received sample (the points on the right) to one of the transmitted symbols (the four points on the left). And we want to be able to do this in realtime, with low latency, for any reasonable condition of the channel. This is typically done by measuring known patterns and using them to extract parameters that are used to characterize the channel. These parameters can be plugged into a channel model and is used to therefore correct the channel variations.

In the above image, the second user is transmitting a completely different sequence of bits but using the same four symbols as the first user. `This is like two persons talking in a room at the same time. How do we separate what each one is saying?` And what happens if the noise is further increased, as in the following image:

<img src="images/apsm_03.png" width="700">

To solve this, one can add more antennas. Since the antennas are not at the same position, each one receives a slightly different signal. Notice how each diagram is rotated differently. Going back to the room analogy, this is equivalent to turning your head, or taking a step closer, to improve the listening of a particular speaker compared to the others.

<img src="images/apsm_04.png" width="700">

While this makes sense, it seems complicated at first to go from the samples on multiple antennas to the individual symbols of multiple users. There are some approaches available, but they have challenges, and `this is where machine learning can help. Instead of modelling the channel, we learn it, creating the model implicitly`. It takes on the task of mapping the received samples to the transmitted symbols efficiently.

For further reading, select the `...` below after the tutorial.

APSM approximates the symbol positions by projecting samples into a set of learned surfaces. These surfaces are learned from the training data. What the method achieves is a functional approximation `f*` of the function `f` based on the known training set `r,b`, as a sum of weighted contributions of some functions, such that we minimize an error figure. This is achieved by means of iterative parallel projections.

<img src="images/apsm_05.png" width="700">

Using a GPU to accelerate such algorithm is a particularly good fit. The APSM projections we just described can be done by a GPU in parallel and very efficiently.
Using a single GPU, we can estimate symbols in as little as 1.5 ms and training the model from a set of pilots (a pilot is a reference and synchronization signal) can be done in the multiple millisecond range. As a reference, a 5G NR frame is 10ms and a subframe 1 ms. This means we have a software solution to do projections in parallel, to process the samples in parallel, and to run reduction in parallel.

Compilation should be completed by now. Return to the jupyter terminal and run the code with the below command from cpp/build:

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

The output `shows a command line representation of the detected symbol errors`. If the error rate is `0.05 %` for the first user in this setup, it worked as expected. (The algorithm detects the symbols for multiple users concurrently.)

Let's look at the timeline of the application in [step 03](03_nsight_systems.ipynb)